#### UNDERSTANDING THE DATA
##### Create a schema 'census' and import table d1, d2 to the schema

In [ ]:
SELECT * FROM census.d1;
SELECT * FROM census.d2;
DESCRIBE census.d1;

| District   | text | YES |
| ---------- | ---- | --- |
| State      | text | YES |
| Area_km2   | text | YES |
| Population | text | YES |

In [ ]:
DESCRIBE census.d2;

| District  | text   | YES |
| --------- | ------ | --- |
| State     | text   | YES |
| Growth    | text   | YES |
| Sex_Ratio | int    | YES |
| Literacy  | double | YES |

In [ ]:
SELECT COUNT(*) FROM census.d1; 
#636
SELECT COUNT(district) FROM census.d1;
#636
SELECT COUNT(DISTINCT district) FROM census.d1;
#630
SELECT COUNT(*) FROM census.d2;
#640
SELECT COUNT(district) FROM census.d2;
#640
SELECT COUNT(DISTINCT district) FROM census.d2;
#634

#### INVESTIGATING DUPLICATES IN census.d1

In [ ]:
DROP TABLE IF EXISTS census.d1_new;
CREATE TABLE census.d1_new
	(SELECT *, 
			ROW_NUMBER() 
            OVER(PARTITION BY district
            ORDER BY district) AS dupp
FROM census.d1);

SELECT * FROM census.d1
WHERE district IN (SELECT district FROM census.d1_new
					WHERE dupp>1)
ORDER BY district;

DROP TABLE census.d1_new;

| District   | State            | Area_km2 | Population |
| ---------- | ---------------- | -------- | ---------- |
| Aurangabad | Maharashtra      | 10,107   | 37,01,282  |
| Aurangabad | Bihar            | 3,305    | 25,40,073  |
| Bijapur    | Karnataka        | 10,498   | 21,77,331  |
| Bijapur    | Chhattisgarh     | 8,530    | 2,55,230   |
| Bilaspur   | Himachal Pradesh | 1,167    | 3,81,956   |
| Bilaspur   | Chhattisgarh     | 8,272    | 26,63,629  |
| Hamirpur   | Uttar Pradesh    | 4,021    | 11,04,285  |
| Hamirpur   | Himachal Pradesh | 1,118    | 4,54,768   |
| Pratapgarh | Uttar Pradesh    | 3,717    | 32,09,141  |
| Pratapgarh | Rajasthan        | 4,449    | 8,67,848   |
| Raigarh    | Maharashtra      | 7,086    | 14,93,984  |
| Raigarh    | Chhattisgarh     | 7,152    | 26,34,200  |

#### INVESTIGATING DUPLICATES IN census.d1

In [ ]:
DROP TABLE IF EXISTS census.d2_new;
CREATE TABLE census.d2_new
	(SELECT *, 
			ROW_NUMBER() 
            OVER(PARTITION BY district
            ORDER BY district) AS dupp
FROM census.d2);

SELECT * FROM census.d2
WHERE district IN (SELECT district FROM census.d2_new
					WHERE dupp>1)
ORDER BY district;

DROP TABLE census.d2_new;

| District   | State            | Growth | Sex_Ratio | Literacy |
| ---------- | ---------------- | ------ | --------- | -------- |
| Aurangabad | Maharashtra      | 0      | 923       | 79.02    |
| Aurangabad | Bihar            | 0      | 926       | 70.32    |
| Bijapur    | Karnataka        | 0      | 960       | 67.15    |
| Bijapur    | Chhattisgarh     | 0      | 984       | 40.86    |
| Bilaspur   | Chhattisgarh     | 0      | 971       | 70.78    |
| Bilaspur   | Himachal Pradesh | 0      | 981       | 84.59    |
| Hamirpur   | Uttar Pradesh    | 0      | 861       | 68.77    |
| Hamirpur   | Himachal Pradesh | 0      | 1,095     | 88.15    |
| Pratapgarh | Uttar Pradesh    | 0      | 998       | 70.09    |
| Pratapgarh | Rajasthan        | 0      | 983       | 55.97    |
| Raigarh    | Maharashtra      | 0      | 959       | 83.14    |
| Raigarh    | Chhattisgarh     | 0      | 991       | 73.26    |

#### d1 + d2 = data WITH TYPE CASTING

In [ ]:
DROP TABLE IF EXISTS census.data;
CREATE TABLE census.data
(
SELECT 
	d2.district, 
	d2.state, 
    CAST(REPLACE(d1.Population,',','') AS UNSIGNED) AS Population, 
    CAST(REPLACE(d1.Area_km2,',','') AS UNSIGNED) AS Area, 
    CAST(REPLACE(d2.Growth,',','') AS DECIMAL(9,2)) AS  Growth, 
    d2.Literacy, 
    d2.Sex_Ratio
FROM census.d1 RIGHT JOIN census.d2
ON d1.district = d2.district AND d1.state = d2.state
ORDER BY state
);

SELECT * FROM census.data;
DESCRIBE census.data;

DELETE FROM census.data WHERE population IS NULL;

| Field      | Type            | Null |
| ---------- | --------------- | ---- |
| district   | text            | YES  |
| state      | text            | YES  |
| Population | bigint unsigned | YES  |
| Area       | bigint unsigned | YES  |
| Growth     | decimal(9,2)    | YES  |
| Literacy   | double          | YES  |
| Sex_Ratio  | int             | YES  |


#### Update table with literacy_sex_growth data

In [ ]:
DROP PROCEDURE IF EXISTS drop_literacy_sex_growth_data;
DELIMITER //

CREATE PROCEDURE drop_literacy_sex_growth_data() BEGIN
IF EXISTS (SELECT * FROM information_schema.COLUMNS WHERE table_schema = 'census' AND COLUMN_NAME = 'Literate') 
THEN ALTER TABLE census.data DROP COLUMN Literate;
END IF;
 
IF EXISTS (SELECT * FROM information_schema.COLUMNS WHERE table_schema = 'census' AND COLUMN_NAME = 'Illiterate') 
THEN ALTER TABLE census.data DROP COLUMN Illiterate;
END IF;
 
IF EXISTS (SELECT * FROM information_schema.COLUMNS WHERE table_schema = 'census' AND COLUMN_NAME = 'Male') 
THEN ALTER TABLE census.data DROP COLUMN Male;
END IF;
 
IF EXISTS (SELECT * FROM information_schema.COLUMNS WHERE table_schema = 'census' AND COLUMN_NAME = 'Female') 
THEN ALTER TABLE census.data DROP COLUMN Female;
END IF;

IF EXISTS (SELECT * FROM information_schema.COLUMNS WHERE table_schema = 'census' AND COLUMN_NAME = 'Prev_Population') 
THEN ALTER TABLE census.data DROP COLUMN Prev_Population;
END IF;

ALTER TABLE census.data ADD COLUMN Literate INTEGER;
ALTER TABLE census.data ADD COLUMN Illiterate INTEGER;
ALTER TABLE census.data ADD COLUMN Male INTEGER;
ALTER TABLE census.data ADD COLUMN Female INTEGER;
ALTER TABLE census.data ADD COLUMN Prev_Population INTEGER;

END//
DELIMITER ;

CALL drop_literacy_sex_growth_data;

UPDATE census.data SET literate = literacy/100 * population;
UPDATE census.data SET illiterate = population - literate;
UPDATE census.data SET male = population/(1+sex_ratio/1000);
UPDATE census.data SET female = population - male;
UPDATE census.data SET Prev_Population = population/(1+growth/100);

DESCRIBE census.data;

| district        | text            | YES |
| --------------- | --------------- | --- |
| state           | text            | YES |
| Population      | bigint unsigned | YES |
| Area            | bigint unsigned | YES |
| Growth          | decimal(9,2)    | YES |
| Literacy        | double          | YES |
| Sex_Ratio       | int             | YES |
| Literate        | int             | YES |
| Illiterate      | int             | YES |
| Male            | int             | YES |
| Female          | int             | YES |
| Prev_Population | int             | YES |